In [1]:
exec(open("./funcs/tool_funcs.py").read())

In [2]:
def shortest_path_n_to_r(distance_matrix, shortest_path_matrix, n, r):
    max_jump_list = []
    for t in range(n):
        m_jump =  distance_matrix[n,t] + shortest_path_matrix[t,r]
        max_jump_list.append(m_jump)
    return np.min(max_jump_list)

def shortest_path_r_to_n(distance_matrix, shortest_path_matrix, n, r):
    max_jump_list = []
    for t in range(n):
        m_jump =  shortest_path_matrix[r,t] + distance_matrix[t, n] 
        max_jump_list.append(m_jump)
    return np.min(max_jump_list)

     
def cal_n_shortest_path(distance_matrix, shortest_path_matrix, n):
    for r in range(n):
        shortest_path_matrix[n,r] = shortest_path_n_to_r(distance_matrix, shortest_path_matrix, n, r)
        shortest_path_matrix[r,n] = shortest_path_r_to_n(distance_matrix, shortest_path_matrix, n, r)
        
    for i in range(n):        
        for j in range(n):
            if i < j:
                shortest_path_matrix[i,j] =  update_shortest_path_ij(distance_matrix, shortest_path_matrix, n, i, j)
                shortest_path_matrix[j,i] =  update_shortest_path_ij(distance_matrix, shortest_path_matrix, n, j, i)
                
def update_shortest_path_ij(distance_matrix, shortest_path_matrix, n, i, j):
    m1 = shortest_path_matrix[i,j]
    m2 =  shortest_path_matrix[i,n] + shortest_path_matrix[n,j] 
    return np.min((m1,m2))

def cal_shortest_path_matrix_algo_1_python(distance_matrix):
    
  
    N = len(distance_matrix)
   
    shortest_path_matrix = np.zeros((N,N))

    shortest_path_matrix[0,1] = distance_matrix[0,1]
    shortest_path_matrix[1,0] = distance_matrix[1,0]
 
    for kk in range(2,N):
        cal_n_shortest_path(distance_matrix, shortest_path_matrix, kk)
    return shortest_path_matrix

In [3]:

X_num_nodes = 300
X_plus_num_nodes = 301
 
 
X_plus_distance_matrix = create_distance_matrix(X_plus_num_nodes)


print(X_plus_distance_matrix[:10, :10])  

[[  0. 612. 457. 440.  32. 132. 630.   2. 880. 742.]
 [396.   0. 125. 946. 404. 382. 227. 546. 794.  66.]
 [777. 669.   0. 549. 210. 786.  41. 980. 868. 696.]
 [214. 536. 228.   0. 459. 801. 661. 938. 781. 346.]
 [390. 890. 999. 668.   0. 774. 159. 710. 922. 123.]
 [ 25. 846.  63. 477. 429.   0. 542. 365. 563. 699.]
 [571. 856. 193. 296. 125. 702.   0. 200. 101. 744.]
 [366. 499.  54. 178. 319. 774. 386.   0. 125.  41.]
 [664. 339. 518. 928. 737. 860. 725. 510.   0. 190.]
 [111. 446. 375. 641. 938. 299. 667. 520. 706.   0.]]


In [4]:
X_distance_matrix = np.zeros((X_num_nodes,X_num_nodes))
for i in range(X_num_nodes):  
    for j in range(X_num_nodes): 
        X_distance_matrix[i,j] = X_plus_distance_matrix[i,j]
 


In [5]:
X_distance_matrix.shape

(300, 300)

In [6]:
X_plus_distance_matrix.shape

(301, 301)

In [7]:
print(X_distance_matrix[:5, :5])  

[[  0. 612. 457. 440.  32.]
 [396.   0. 125. 946. 404.]
 [777. 669.   0. 549. 210.]
 [214. 536. 228.   0. 459.]
 [390. 890. 999. 668.   0.]]


In [8]:
print(X_plus_distance_matrix[:5, :5])  

[[  0. 612. 457. 440.  32.]
 [396.   0. 125. 946. 404.]
 [777. 669.   0. 549. 210.]
 [214. 536. 228.   0. 459.]
 [390. 890. 999. 668.   0.]]


In [9]:
# Test cold-start calculation of the APSP matrix by Floyd-Warshall algorithm.


start = time.time()
X_plus_APSP_matrix_Floyd_Warshall_python = floyd_warshall(X_plus_distance_matrix)
end = time.time()
time_used = end - start
time_used = np.round(time_used, 3)
print(f"Cold start calculation of the APSP matrix by Floyd-Warshall algorithm, time_used: {time_used}s.")


Cold start calculation of the APSP matrix by Floyd-Warshall algorithm, time_used: 28.937s.


In [10]:


X_APSP_matrix = floyd_warshall(X_distance_matrix)

In [11]:
X_APSP_matrix.shape

(300, 300)

In [12]:
def cal_APSP_matrix_algo_1_warm_start(X_plus_distance_matrix, X_APSP_matrix):

    N = len(X_APSP_matrix)
    
    N_plus_one = N + 1
   
    X_plus_APSP_matrix = np.zeros((N_plus_one, N_plus_one))
    
    for i in range(N):
        for j in range(N):
            X_plus_APSP_matrix[i,j] = X_APSP_matrix[i,j]
            
    cal_n_shortest_path(X_plus_distance_matrix, X_plus_APSP_matrix, N)
 
    return X_plus_APSP_matrix

In [13]:
# Test warm-start calculation of the APSP matrix by Algorithm 1.
 
start = time.time()
X_plus_APSP_matrix_algo_1_warm = cal_APSP_matrix_algo_1_warm_start(X_plus_distance_matrix, X_APSP_matrix)
end = time.time()
time_used = end - start
time_used = np.round(time_used, 3)

print(f"Warm-start calculation of the APSP matrix by Algorithm 1, time_used: {time_used}s.")

Warm-start calculation of the APSP matrix by Algorithm 1, time_used: 0.686s.


In [14]:

print(np.allclose(X_plus_APSP_matrix_Floyd_Warshall_python, X_plus_APSP_matrix_algo_1_warm))
print(np.sum(np.abs(X_plus_APSP_matrix_Floyd_Warshall_python - X_plus_APSP_matrix_algo_1_warm)))


True
0.0


In [15]:
# Therefore, we can conclude warm-start of Algorithm 1 is faster than cold-start of Floyd-Warshall algorithm,
# for calculating the All Pairs Shortest Path (APSP) matrix.
